## **Machine Learning - Getaround project**

# Préparation des données

In [1]:
from collections import Counter

import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import Lasso, LinearRegression, LogisticRegression, Ridge
from sklearn.metrics import accuracy_score, classification_report, f1_score, mean_squared_error, r2_score, silhouette_score
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import class_weight
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, LeakyReLU
from tensorflow.keras.models import Sequential
from xgboost import XGBRegressor

from tensorflow import keras


In [2]:
# Chargement des données
df_price = pd.read_csv('/content/get_around_pricing_project.csv')

In [3]:
#On va garder que les marques avec plus de 15 véhicules
threshold = 15
value_counts = df_price['model_key'].value_counts()
to_remove = value_counts[value_counts < threshold].index
df_price = df_price[~df_price['model_key'].isin(to_remove)]

In [6]:
# Marques conservées
brands_retained = df_price['model_key'].unique()
print("marques conservées :")
print(brands_retained)

marques conservées :
['Citroën' 'Peugeot' 'PGO' 'Renault' 'Audi' 'BMW' 'Mercedes' 'Opel'
 'Volkswagen' 'Ferrari' 'Maserati' 'Mitsubishi' 'Nissan' 'SEAT' 'Subaru'
 'Toyota']


In [ ]:
# Suppression de colonnes non pertinentes
df_price.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
# Séparation des caractéristiques et de la cible
X = df_price.drop('rental_price_per_day', axis=1)
Y = df_price['rental_price_per_day']

# Séparation en ensembles d'entraînement et de test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Identification des caractéristiques numériques et catégorielles
num_features = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_features = X_train.select_dtypes(include=['object']).columns

In [ ]:
# Pipeline pour les transformations
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(
    steps=[
        ('encoder', OneHotEncoder(drop='first'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ]
)

# Premier modèle - Regression linéaire

In [ ]:
# Modèle de régression linéaire
model_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model_lr.fit(X_train, Y_train)
Y_pred_train = model_lr.predict(X_train)
Y_pred_test = model_lr.predict(X_test)

print("Linear Regression - R2 score on training set:", r2_score(Y_train, Y_pred_train))
print("Linear Regression - R2 score on test set:", r2_score(Y_test, Y_pred_test))


Linear Regression - R2 score on training set: 0.6699435611245842
Linear Regression - R2 score on test set: 0.6600718859204879


In [ ]:
 # Ajustement des hyperparamètres avec Ridge
model_ridge = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

params_ridge = {
    'regressor__alpha': [0.01, 0.1, 1, 10]
}

grid_ridge = GridSearchCV(model_ridge, param_grid=params_ridge, cv=10, scoring='r2')
grid_ridge.fit(X_train, Y_train)

print("Ridge - Meilleur score R2 en validation croisée:", grid_ridge.best_score_)
print("Ridge - Meilleurs hyperparamètres:", grid_ridge.best_params_)

Ridge - Meilleur score R2 en validation croisée: 0.6603210885098869
Ridge - Meilleurs hyperparamètres: {'regressor__alpha': 10}


In [ ]:
# Création du pipeline de régression Lasso
model_lasso = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Lasso())
])

# Définition de la grille de paramètres pour GridSearchCV
param_grid = {
    'regressor__alpha': [0.001, 0.01, 0.02, 0.1, 1, 10]
}

# Configuration et exécution de la recherche d'hyperparamètres
grid_search = GridSearchCV(model_lasso, param_grid, cv=10, scoring='r2', return_train_score=True)
grid_search.fit(X_train, Y_train)

# Résultats
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleur score R2 de validation croisée : ", grid_search.best_score_)

# Prédiction et évaluation avec le meilleur modèle trouvé
Y_pred_train = grid_search.predict(X_train)
Y_pred_test = grid_search.predict(X_test)

print("R2 score sur l'ensemble d'entraînement :", r2_score(Y_train, Y_pred_train))
print("R2 score sur l'ensemble de test :", r2_score(Y_test, Y_pred_test))


Meilleurs paramètres :  {'regressor__alpha': 0.02}
Meilleur score R2 de validation croisée :  0.6586467283367599
R2 score sur l'ensemble d'entraînement : 0.6683487192388415
R2 score sur l'ensemble de test : 0.6565560010658167


Le modèle de régression linéaire lasso présente des performances équilibrées mais qui pourraient être meilleures en terme de R2.

# Second modèle - Random Forest

In [ ]:
# Création du pipeline avec Random Forest
model_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

# Définition de la grille de paramètres
param_grid_rf = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5],
    'regressor__min_samples_leaf': [1, 2]
}

# Configuration de GridSearchCV
grid_search_rf = GridSearchCV(model_rf, param_grid_rf, cv=5, scoring='r2', return_train_score=True, verbose=2)
grid_search_rf.fit(X_train, Y_train)

# Meilleurs paramètres
print("Meilleurs paramètres : ", grid_search_rf.best_params_)
print("Meilleur score R2 de validation croisée : ", grid_search_rf.best_score_)

# Prédiction et évaluation avec le meilleur modèle trouvé
Y_pred_train_rf = grid_search_rf.predict(X_train)
Y_pred_test_rf = grid_search_rf.predict(X_test)

print("Random Forest - R2 score sur l'ensemble d'entraînement :", r2_score(Y_train, Y_pred_train_rf))
print("Random Forest - R2 score sur l'ensemble de test :", r2_score(Y_test, Y_pred_test_rf))


NameError: name 'Pipeline' is not defined

# Troisième modèle XGBOOST

Le modèle de random forest regressor surapprend beaucoup, malgrés un bon R2 en en entrainenement nous allons nous tourner vers un autre modèle.


In [ ]:
# Modèle XGBoost
model_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

params_xgb = {
    'regressor__max_depth': [3, 5, 7],
    'regressor__n_estimators': [50, 100, 200],
    'regressor__learning_rate': [0.01, 0.1, 0.2, 0.3]
}

grid_xgb = GridSearchCV(model_xgb, param_grid=params_xgb, cv=10, scoring='r2', verbose=1)
grid_xgb.fit(X_train, Y_train)

print("XGBoost - Meilleur score R2 en validation croisée:", grid_xgb.best_score_)
print("XGBoost - Meilleurs hyperparamètres:", grid_xgb.best_params_)

# Évaluation du modèle final XGBoost sur l'ensemble de test
final_model = grid_xgb.best_estimator_
Y_pred_test_final = final_model.predict(X_test)
print("XGBoost - R2 score sur l'ensemble de test:", r2_score(Y_test, Y_pred_test_final))

Fitting 10 folds for each of 36 candidates, totalling 360 fits
XGBoost - Meilleur score R2 en validation croisée: 0.7266801355059139
XGBoost - Meilleurs hyperparamètres: {'regressor__learning_rate': 0.2, 'regressor__max_depth': 3, 'regressor__n_estimators': 200}
XGBoost - R2 score sur l'ensemble de test: 0.7332491656908411


Le modèle Xgboost présente des performances correctes et équilibrées nous allons donc nous tourner vers ce modèle.

In [ ]:
grid_xgb.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['mileage', 'engine_power'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  Index(['model_key', 'fuel', 'paint_color', 'car_type'], dtype='object'))])),
                ('regressor',
                 XGBRegressor(base_score=None, boos...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.2,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=3, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=200, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [ ]:
# Enregistrement de la pipeline complète
joblib.dump(grid_xgb.best_estimator_, 'final_model.joblib')

['final_model.joblib']

In [ ]:
# Charger le modèle depuis le fichier .joblib
model = joblib.load("/content/final_model.joblib")

# Maintenant, vous pouvez inspecter le modèle
print(model)

NameError: name 'joblib' is not defined